In [1]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"

/media/paulati/Nuevo vol/paula/dh/2021/dsad_2021/common
default checking
Running command `conda list`... ok
jupyterlab=2.2.6 already installed
pandas=1.1.5 already installed
bokeh=2.2.3 already installed
seaborn=0.11.0 already installed
matplotlib=3.3.2 already installed
ipywidgets=7.5.1 already installed
pytest=6.2.1 already installed
chardet=4.0.0 already installed
psutil=5.7.2 already installed
scipy=1.5.2 already installed
statsmodels=0.12.1 already installed
scikit-learn=0.23.2 already installed
xlrd=2.0.1 already installed
Running command `conda install --yes nltk=3.5.0`... ok
Solving environment: ...working... done

# All requested packages already installed.


unidecode=1.1.1 already installed
pydotplus=2.0.2 already installed
pandas-datareader=0.9.0 already installed
flask=1.1.2 already installed


---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


## Introducción

Data wrangling es el proceso de limpieza y unificación de conjuntos de datos desordenados y complejos para facilitar su acceso, exploración, análisis o modelización posterior.

Las tareas que involucra son
* Limpieza de datos
* Eliminación de registros duplicados
* Transformación de datos
* Discretización de variables
* Detección y filtro de outliers
* Construcción de variables dummies

Pandas provee métodos para llevar a cabo estas tareas, y en esta práctica repasaremos algunos de estos métodos.

## Dataset

En esta clase usaremos un dataset con info de películas que disponibiliza datos de movielens (https://movielens.org/).

https://grouplens.org/datasets/movielens/

http://files.grouplens.org/datasets/movielens/ml-latest-small.zip

Este conjunto de datos está conformado por varios archivos:
* **movies**: idPelicula, título y género; 

donde cada registro tiene los datos de una película

* **ratings**: idUsuario, idPelicula, rating, fecha; 

donde cada registro tienen la calificación otorgada por un usuario a una película

* **tags**: idUsuario, idPelicula, tag, fecha; 

donde cada registro tienen el tag que asignó un usuario a una película

## Imports

In [2]:
import pandas as pd
import numpy as np
import re

## Ejercicio 1 - Importar datos

Leamos los datos de movies, ratings y tags desde los archivos
* ../Data/movies.csv
* ../Data/ratings.csv
* ../Data/tags.csv

en las variables 
* data_movies
* data_ratings
* data_tags

Veamos cuántos registros hay en cada DataFrame y de qué tipos son los datos de cada columna. 

Veamos los primeros registros de cada DataFrame para verificar que los datos fueron importados correctamente.

In [3]:
data_movies = pd.read_csv("../Data/movies.csv", sep=",")
print(data_movies.dtypes)
data_movies.head(3)

movieId     int64
title      object
genres     object
dtype: object


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [4]:
data_ratings = pd.read_csv("../Data/ratings.csv", sep=",")
print(data_ratings.dtypes)
data_ratings.head(3)

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224


In [5]:
data_tags = pd.read_csv("../Data/tags.csv", sep=",")
print(data_tags.dtypes)
data_tags.head(3)

userId        int64
movieId       int64
tag          object
timestamp     int64
dtype: object


,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992


## Ejercicio 2 - duplicated, drop_duplicates

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html

**2.a** Analicemos los ratings asignados por los usuarios a las películas, y detectemos aquellos usuarios que calificaron más de una película.

**2.b** Veamos si hay usuarios que calificaron más de una vez a la misma película

**2.c** Creemos un dataframe donde cada usuario haya calificado exactamente una película

In [6]:
# 2.a

user_many_ratings = data_ratings.duplicated(subset = ["userId"], keep = "first")
#print(user_many_ratings)
any(user_many_ratings)

True

In [7]:
# 2.b

user_one_movie_many_ratings = data_ratings.duplicated(subset = ["userId", "movieId"], keep = "first")
any(user_one_movie_many_ratings)

False

In [8]:
# 2.c

user_one_rating = data_ratings.drop_duplicates(subset = ["userId"], keep = "first")

print(data_ratings.shape)

print(user_one_rating.shape)


(100836, 4)
(610, 4)


In [9]:
len(user_one_rating.userId.unique())

610

## Ejercicio 3 - replace

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.replace.html

El campo `genre` del DataFrame data_movies tiene una lista de géneros separadas por el caracter "|"

**3.a** Una lista de valores viejos por un valor nuevo

Vamos a buscar los valores de los registros que tengan "Fantasy" como alguno sus géneros y reemplazarlos por "Fantasy"

**3.b** Una lista de valores viejos por una lista de valores nuevos

Tomamos 10 valores del campo género al azar y construimos una lista con ellos, y otra con el último género que figura en cada elemento de la primera lista. Usamos esas dos listas para reemplazar unos valores por otros.

**3.c** Un diccionario que defina los mapeos

Con las dos listas del paso anterior, construimos un diccionario que pasamos como argumento al método replace para definir algunos reemplazos.


In [10]:
# 3.a

fantasy_pattern = "Fantasy"
fantasy_regex = re.compile(fantasy_pattern)
fantasy_matches = data_movies.genres.apply(lambda x: x if x is np.NaN else fantasy_regex.search(x))
fantasy_mask = fantasy_matches.notnull()
fantasy_genres = fantasy_matches.loc[fantasy_mask]
#print(fantasy_genres)
fantasy_genres.iloc[0].string
genres_to_replace = [x.string for x in fantasy_genres]
print(len(genres_to_replace))
genres_to_replace_unique = list(set(genres_to_replace))
print(len(genres_to_replace_unique))

data_genres_3a = data_movies.replace(genres_to_replace_unique, "Fantasy")
data_genres_3a
#data_movies

779
264


,movieId,title,genres
0,1,Toy Story (1995),Fantasy
1,2,Jumanji (1995),Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Fantasy
9738,193583,No Game No Life: Zero (2017),Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [11]:
# 3.b

sample_genres = np.random.choice(data_movies.genres.unique(), 10, replace= False)
print("géneros originales: ", sample_genres)

def get_last_genre(value):
    parts = value.split("|")
    last_index = len(parts) - 1
    result = parts[last_index]
    return result

sample_genres_replacement = [get_last_genre(x) for x in sample_genres]
print("géneros de reemplazo: ", sample_genres_replacement)

data_genres_3b = data_movies.replace(sample_genres, sample_genres_replacement)
data_genres_3b

géneros originales:  ['Animation' 'Thriller' 'Adventure|Animation|Children|Comedy|Musical'
 'Action|Drama|Horror' 'Action|Adventure|Drama|Sci-Fi|Thriller'
 'Horror|Mystery|Sci-Fi' 'Action|Animation|Children|Fantasy'
 'Drama|Fantasy|Sci-Fi' 'Drama|Horror|Romance|Thriller'
 'Comedy|Crime|Mystery|Thriller']
géneros de reemplazo:  ['Animation', 'Thriller', 'Musical', 'Horror', 'Thriller', 'Sci-Fi', 'Fantasy', 'Sci-Fi', 'Thriller', 'Thriller']


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [12]:
# 3.c
keys = sample_genres
values = sample_genres_replacement
replacement_dict = dict(zip(keys, values))
replacement_dict

{'Animation': 'Animation',
 'Thriller': 'Thriller',
 'Adventure|Animation|Children|Comedy|Musical': 'Musical',
 'Action|Drama|Horror': 'Horror',
 'Action|Adventure|Drama|Sci-Fi|Thriller': 'Thriller',
 'Horror|Mystery|Sci-Fi': 'Sci-Fi',
 'Action|Animation|Children|Fantasy': 'Fantasy',
 'Drama|Fantasy|Sci-Fi': 'Sci-Fi',
 'Drama|Horror|Romance|Thriller': 'Thriller',
 'Comedy|Crime|Mystery|Thriller': 'Thriller'}

In [13]:
data_genres_3c = data_movies.replace(replacement_dict)
data_genres_3c

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


## Ejercicio 4 - cut

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.cut.html

El método `cut` nos permite asignar categorías a intervalos numéricos.

Creemos la columna de categorías (rating_label_cut) asociadas al campo rating de data_ratings de este modo:

* mala, para puntajes menores a 3;

* regular, para mayor igual a 3 y  menor que 4;

* buena para puntaje mayor o igual que 4


In [14]:
# valores de corte:
bins_rating = [0, 3, 4, 5.1]
labels_rating = ["mala", "regular", "buena"]

# Obtengo una lista de intervalos
# right=False indice que el extremo derecho del intervalo no está incluído en la categoría
categories = pd.cut(data_ratings.rating, bins_rating, labels = labels_rating, right=False)
#print(categories)
data_ratings["rating_label_cut"] = categories
data_ratings.head(10)

,userId,movieId,rating,timestamp,rating_label_cut
0,1,1,4.0,964982703,buena
1,1,3,4.0,964981247,buena
2,1,6,4.0,964982224,buena
3,1,47,5.0,964983815,buena
4,1,50,5.0,964982931,buena
5,1,70,3.0,964982400,regular
6,1,101,5.0,964980868,buena
7,1,110,4.0,964982176,buena
8,1,151,5.0,964984041,buena
9,1,157,5.0,964984100,buena


##  Ejercicio 5 - quantile

Calculemos los cuartilos del campo timestamp de la tabla data_ratings y asociemos las etiquetas:
* muy antigua, para el primer cuartilo
* antigua, para el segundo cuartilo
* pasada, para el tercer cuartilo
* reciente, para el cuarto cuartilo

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.quantile.html

In [15]:
q1 = data_ratings.timestamp.quantile(0.25)
q2 = data_ratings.timestamp.quantile(0.5)
q3 = data_ratings.timestamp.quantile(0.75)
q4 = data_ratings.timestamp.quantile(1)

bins_timestamp = [0, q1, q2, q3, q4]
labels_timestamp = ["muy antigua", "antigua", "pasada", "reciente"]

timestamp_categories = pd.cut(data_ratings.timestamp, bins_timestamp, labels = labels_timestamp, right=False)

data_ratings["timestamp_category"] = timestamp_categories

print(data_ratings.loc[data_ratings.timestamp_category == "reciente", ].shape)
print(data_ratings.loc[data_ratings.timestamp_category == "pasada", ].shape)
print(data_ratings.loc[data_ratings.timestamp_category == "antigua", ].shape)
print(data_ratings.loc[data_ratings.timestamp_category == "muy antigua", ].shape)

data_ratings.head(10)

(25208, 6)
(25209, 6)
(25210, 6)
(25208, 6)


,userId,movieId,rating,timestamp,rating_label_cut,timestamp_category
0,1,1,4.0,964982703,buena,muy antigua
1,1,3,4.0,964981247,buena,muy antigua
2,1,6,4.0,964982224,buena,muy antigua
3,1,47,5.0,964983815,buena,muy antigua
4,1,50,5.0,964982931,buena,muy antigua
5,1,70,3.0,964982400,regular,muy antigua
6,1,101,5.0,964980868,buena,muy antigua
7,1,110,4.0,964982176,buena,muy antigua
8,1,151,5.0,964984041,buena,muy antigua
9,1,157,5.0,964984100,buena,muy antigua


##  Ejercicio 6 - get_dummies

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html

Vamos a crear ahora variables dummy para representar el campo timestamp_category del DataFrame data_ratings, y vamos a reemplazar ese campo por las variables dummies.


In [16]:
dummies_timestamp_category = pd.get_dummies(data_ratings.timestamp_category, prefix='timestamp_category', drop_first=True)

#print(dummies_timestamp_category)

# agregamos las columnas dummies
data_ratings_with_dummy = data_ratings.join(dummies_timestamp_category)

#data_ratings_with_dummy

# quitamos la columna timestamp_category que ahora queda representada por las dummies
result = data_ratings_with_dummy.drop(labels = ["timestamp_category"], axis="columns")
result.head(5)

,userId,movieId,rating,timestamp,rating_label_cut,timestamp_category_antigua,timestamp_category_pasada,timestamp_category_reciente
0,1,1,4.0,964982703,buena,0,0,0
1,1,3,4.0,964981247,buena,0,0,0
2,1,6,4.0,964982224,buena,0,0,0
3,1,47,5.0,964983815,buena,0,0,0
4,1,50,5.0,964982931,buena,0,0,0
